<a href="https://colab.research.google.com/github/porekhov/drug_design_2024/blob/main/rdkit_formats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install Conda Colab and rdkit**

%%capture
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge rdkit -y
!pip install py3Dmol

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from IPython.display import display
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions

In [ ]:
# RDKit docs: https://rdkit.readthedocs.io/en/latest/index.html

smiles = 'O=C(C)Oc1ccccc1C(=O)O'

# create an object molecule from the smiles string
mol = Chem.MolFromSmiles(smiles)

# create an object molecule from the mol file
#Chem.MolFromMolFile('molecule.mol')

# show the structural formula
display(Draw.MolToImage(mol))

mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, randomSeed = 12345)
AllChem.UFFOptimizeMolecule(mol)

print(Chem.MolToMolBlock(mol))
# print mol block to a file
print(Chem.MolToMolBlock(mol), file=open('aspirin.mol','w+'))

In [ ]:
import py3Dmol

view = py3Dmol.view(width=400, height=300)
view.addModel(open('aspirin.mol', 'r').read(),'mol')
view.setStyle({'model': -1}, {"stick": {}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()

In [ ]:
mols = Chem.SDMolSupplier('library.sdf')
print('Number of molecules in the SDF file:', len(mols))

for mol in mols:
    print(mol.GetNumAtoms())
    print(Chem.MolToSmiles(mol))
    display(Draw.MolToImage(mol))

In [ ]:
ibu_smi = 'CC(C)Cc1ccc(cc1)[C@@H](C)C(=O)O'

ibu_mol = Chem.MolFromSmiles(ibu_smi)
ibu_mol = Chem.AddHs(ibu_mol)

# show the structural formula
display(Draw.MolToImage(ibu_mol))

In [ ]:
# generate multiple (10) conformers for ibu_mol
conf_ids = AllChem.EmbedMultipleConfs(ibu_mol,
                                  clearConfs=True,
                                  numConfs=10,
                                  pruneRmsThresh=0.1)

# optimize structure of each conformer
for conf_id in conf_ids:
    AllChem.UFFOptimizeMolecule(ibu_mol, confId=conf_id)

# create a pattern (benzene ring) to align
patt = Chem.MolFromSmarts('c1ccccc1')
patt

In [ ]:
match = ibu_mol.GetSubstructMatch(patt)
print(match)
DrawingOptions.includeAtomNumbers=True
ibu_mol

In [ ]:
# align molecules using the selected pattern (benzene ring)
AllChem.AlignMolConformers(ibu_mol, atomIds=match)

# write an sdf file with all conformations
writer = Chem.SDWriter('ibu.sdf')

for conf_id in conf_ids:
    writer.write(ibu_mol, confId=conf_id)
writer.close()

In [ ]:
# show conformers from sdf using py3Dmol
# open sdf, iterate over the conformations,
# shown each molecule with the different color

view = py3Dmol.view(width=400, height=300)
view.setBackgroundColor('white')
colors=('cyanCarbon','redCarbon','blueCarbon')

suppl = Chem.SDMolSupplier('ibu.sdf')

for idx, mol in enumerate(suppl):
    if mol:
        mol_block = Chem.MolToMolBlock(mol)
        view.addModel(mol_block, 'sdf', {'index': idx, 'keepH': True, 'doAssembly': True})
        view.setStyle({'model': idx}, {'stick': {'colorscheme':colors[idx%len(colors)]}})

view.zoomTo()
view.setBackgroundColor('white')
view.show()